# Overlaying Foreground on Background Image and Generating Mask
----

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import math, random, cv2, glob, os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import zipfile
from tqdm import tqdm

In [0]:
os.chdir('gdrive/My Drive/EVA4/DepthMaskDataset')

In [0]:
bg_path = "./data/bg"
fg_path = "./data/fg"
fg_mask_path = "./data/fg_mask"

In [0]:
def overlay_fg_over_bg(bg_path, fg_path, fg_mask_path, zip_file_name='custom_depth_mask.zip'):
  index = 1
  total_iter = 20

  zf = zipfile.ZipFile(zip_file_name, mode='w', compression=zipfile.ZIP_STORED)
  label_info = open('data_label.txt', 'w+')

  for bg_index, bg_filename in enumerate(tqdm(os.listdir(bg_path))):
      bg_name = bg_filename.split('.jpg')[0]

      fg_bg_label_info = open(f'fg_bg_label.txt', 'w+')
                  
      for fg_index, fg_filename in enumerate(os.listdir(fg_path)):
          fg_name = fg_filename.split('.png')[0]

          for itr in range(0, total_iter):
              fg = Image.open(os.path.join(fg_path, fg_filename))
              fg_mask = Image.open(os.path.join(fg_mask_path, os.listdir(fg_mask_path)[fg_index])).convert('L')
              
              # Flip image and mask
              fg_flipped = fg.transpose(Image.FLIP_LEFT_RIGHT)
              fg_mask_flipped = fg_mask.transpose(Image.FLIP_LEFT_RIGHT)
              
              for (img, mask) in [(fg, fg_mask), (fg_flipped, fg_mask_flipped)]:
                  
                  fg_bg = Image.open(os.path.join(bg_path, bg_filename))
                  
                  x_pos_range = (0, fg_bg.size[0])
                  y_pos_range = (0, fg_bg.size[1])
                  
                  # Randomly generate (x,y) position and scaling ratio
                  x_pos = random.randrange(*x_pos_range)
                  y_pos = random.randrange(*y_pos_range)
                  # scale_ratio = max(0.6, round(random.random(), 2))

                  # # Resize foreground and its mask 
                  # new_size = (math.ceil(img.size[0] * scale_ratio), math.ceil(img.size[1] * scale_ratio))
                  # img_resized = img.resize(new_size)
                  # mask_resized = mask.resize(new_size)

                  fg_bg.paste(img, (x_pos, y_pos), img) #(img_resized, (x_pos, y_pos), img_resized)

                  fg_bg_mask = Image.new('L', fg_bg.size)
                  ##fg_mask = mask_resized.convert('L')
                  fg_bg_mask.paste(mask, (x_pos, y_pos), mask) #(mask_resized, (x_pos, y_pos), mask_resized)

                  # Save the background and the mask as temp .jpg files
                  fg_bg.save('fg_bg.jpg', optimize=True, quality=50)
                  fg_bg_mask.save('fg_bg_mask.jpg', optimize=True, quality=50)
                  
                  zf.write('fg_bg.jpg', f'data/fg_bg/{bg_name}/{bg_name}_{fg_name}_{index:06d}.jpg')
                  zf.write('fg_bg_mask.jpg', f'data/fg_bg_mask/{bg_name}/{bg_name}_{fg_name}_{index:06d}.jpg')
                  
                  label_info.write(f'data/fg_bg/{bg_name}/{bg_name}_{fg_name}_{index:06d}.jpg\tdata/fg_bg_mask/{bg_name}/{bg_name}_{fg_name}_{index:06d}.jpg\t{x_pos}\t{y_pos}\t{img.size[0]}\t{img.size[1]}\n')
                  fg_bg_label_info.write(f'data/fg_bg/{bg_name}/{bg_name}_{fg_name}_{index:06d}.jpg\tdata/fg_bg_mask/{bg_name}/{bg_name}_{fg_name}_{index:06d}.jpg\t{x_pos}\t{y_pos}\t{img.size[0]}\t{img.size[1]}\n')

                  index += 1
                  
                  # Close open files
                  fg_bg.close()
              
              fg.close()
              fg_mask.close()
    
      fg_bg_label_info.close()
      
      # write the labels file to zip
      zf.write('fg_bg_label.txt', f'data/labels/{bg_name}_label_info.txt')
      
  label_info.close()

  # write the labels file to zip
  zf.write('data_label.txt')

  # Close the zip, else it gets corrupted
  zf.close()    

In [0]:
overlay_fg_over_bg(bg_path, fg_path, fg_mask_path, 'depth_mask_dataset.zip')

100%|██████████| 100/100 [1:49:42<00:00, 65.83s/it]
